<a href="https://colab.research.google.com/github/VGokulPillai/Fault-Classification-Using-ML/blob/main/EnergyClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import seaborn as sns


dataset = pd.read_csv('/content/Energy.csv')
X = dataset[['Bus1','Bus2', 'Bus3', 'Bus4']]
y = dataset['Symmetrical or Unsymmetrical']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
feature_importances = regressor.feature_importances_

X_train_opt = X_train.copy()
removed_columns = pd.DataFrame()
models = []
r2s_opt = []


for i in range(0, 5):
    regressor.fit(X_train_opt, y_train)
    feature_importances = regressor.feature_importances_
    accuracies = cross_val_score(estimator=regressor,
                                 X=X_train_opt,
                                 y=y_train, cv=5,
                                 scoring='r2')
    r2s_opt = np.append(r2s_opt, accuracies.mean())
    models = np.append(models, ", ".join(list(X_train_opt)))

feature_selection = pd.DataFrame({'Features': models, 'r2 Score': r2s_opt})
feature_selection.head()

X_train_best = X_train[['Bus1','Bus2', 'Bus3', 'Bus4']]
X_test_best = X_test[['Bus1','Bus2', 'Bus3', 'Bus4']]
regressor.fit(X_train_best, y_train)
pickle.dump(regressor,open('Finalmodel.pkl','wb'))
model = pickle.load(open('Finalmodel.pkl','rb'))

accuracies = cross_val_score(estimator = regressor, X = X_train_best,y = y_train, cv = 10, scoring = 'r2')
accuracy = accuracies.mean()
#print('r2 = {}'.format(accuracy))

from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
y_pred = regressor.predict(X_test_best)
explained_variance_score = explained_variance_score(y_test, y_pred)
mean_squared_error = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
#print('explained variance = {}'.format(explained_variance_score))
#print('mse = {}'.format(mean_squared_error))
#print('r2 = {}'.format(r_squared))
Bus1  = float(input("Enter Bus1 Current Fault: "))
Bus2  = float(input("Enter Bus2 Current Fault: "))
Bus3  = float(input("Enter Bus3 Current Fault: "))
Bus4  = float(input("Enter Bus4 Current Fault: "))
result = model.predict([[Bus1,Bus2,Bus3,Bus4]])
if result==1:
  print("The fault is classified into Symmetrical based on the given values and using ML program")
else:
  print("The fault is classified into Unsymmetrical based on the given values and using ML program")


Enter Bus1 Current Fault: 10.374
Enter Bus2 Current Fault: 8.58
Enter Bus3 Current Fault: 8.364
Enter Bus4 Current Fault: 8.063
The fault is classified into Symmetrical based on the given values and using ML program
